In [2]:
#!pip install rouge-score
!pip install rouge-score
!pip install pycocoevalcap

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
#import tf_tf_rouge_l #rouge_l_tensorflow

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Metrics

## BLEU-1 (see 1-gram below)
### Python implementation wihtin NLTK

In [6]:
import nltk as nltk
import nltk.translate as ntrans
import nltk.translate.bleu_score as ntbs

print("Sentence BLEU Score: Perfect match")
reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
candidate = ['this', 'is', 'a', 'test']
score = ntbs.sentence_bleu(reference, candidate)
print(score)

print("\nSentence BLEU Score: Individual N-GRAM Score")
reference = [['this', 'is', 'a', 'test']]
candidate = ['this', 'is', 'a', 'test']
print('Individual 1-gram - BLEU-1: %f' % ntbs.sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Individual 2-gram - BLEU-2: %f' % ntbs.sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
print('Individual 3-gram - BLEU-3: %f' % ntbs.sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
print('Individual 4-gram - BLEU-4: %f' % ntbs.sentence_bleu(reference, candidate, weights=(0, 0, 0, 1)))

print("\nSentence BLEU Score: Cumulative N-GRAM Score")
reference = [['this', 'is', 'small', 'test']]
candidate = ['this', 'is', 'a', 'test']
score = ntbs.sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))
print(score)

print("\nCorpus (more than one reference sentence) BLEU Score")
references = [[['this', 'is', 'a', 'small', 'test'], ['this', 'is', 'test']]]
candidates = [['this', 'is', 'a', 'test']]
score = ntbs.corpus_bleu(references, candidates)
print(score)

Sentence BLEU Score: Perfect match
1.0

Sentence BLEU Score: Individual N-GRAM Score
Individual 1-gram - BLEU-1: 1.000000
Individual 2-gram - BLEU-2: 1.000000
Individual 3-gram - BLEU-3: 1.000000
Individual 4-gram - BLEU-4: 1.000000

Sentence BLEU Score: Cumulative N-GRAM Score
1.0547686614863434e-154

Corpus (more than one reference sentence) BLEU Score
9.280167055464391e-78


/home/ths/miniconda3/envs/aida/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ths/miniconda3/envs/aida/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [7]:
reference = ["police killed the gunman".split(" "), "police kill the gunman".split(" ")]
candidate1 = "police kill the gunman".split(" ")
candidate2 = "the gunman kill police ".split(" ")
score1 = ntbs.sentence_bleu(reference, candidate1, weights=(1, 0, 0, 0))
score2 = ntbs.sentence_bleu(reference, candidate2, weights=(1, 0, 0, 0))

(reference, candidate1, score1), (reference, candidate2, score2)

(([['police', 'killed', 'the', 'gunman'], ['police', 'kill', 'the', 'gunman']],
  ['police', 'kill', 'the', 'gunman'],
  1.0),
 ([['police', 'killed', 'the', 'gunman'], ['police', 'kill', 'the', 'gunman']],
  ['the', 'gunman', 'kill', 'police', ''],
  0.8))

In [5]:
import pycocoevalcap.bleu.bleu as pyco_bleu
reference = ["police killed the gunman"]
candidate1 = "police kill the gunman"
candidate2 = "the gunman kill police"

bleu_scorer = pyco_bleu.BleuScorer(n=2)
bleu_scorer.compute_score(option="closest", verbose=2)

{'testlen': 0, 'reflen': 0, 'guess': [0, 0], 'correct': [0, 0]}
ratio: 1e-06


([0.0, 0.0], [[], []])

In [6]:
bleu_scorer += (candidate1, reference)
bleu_scorer += (candidate2, reference)

score, scores = bleu_scorer.compute_score(option="closest", verbose=2)
score

{'reflen': [4], 'testlen': 4, 'guess': [4, 3, 2, 1], 'correct': [3, 1, 0, 0]} 4
{'reflen': [4], 'testlen': 4, 'guess': [4, 3, 2, 1], 'correct': [3, 1, 0, 0]} 4
{'testlen': 8, 'reflen': 8, 'guess': [8, 6], 'correct': [6, 2]}
ratio: 0.9999999998750002


[0.7499999998125003, 0.49999999986458354]

In [16]:
"""
Usage of metrics
model.compile(optimizer=Adam(lr=self._learning_rate, clipnorm=5.0),
                      loss=categorical_crossentropy_from_logits,
                      metrics=[categorical_accuracy_with_variable_timestep])
"""
import src.models.metrics as met
met.categorical_accuracy_with_variable_timestep

<function src.models.metrics.categorical_accuracy_with_variable_timestep(y_true, y_pred)>

## ROUGE-L

In [3]:
import rouge_score.rouge_scorer as rouge_scorer

reference = "police killed the gunman"
candidate1 = "police kill the gunman"
candidate2 = "the gunman kill police"
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=False)
scores1 = scorer.score(reference, candidate1)
scores2 = scorer.score(reference, candidate2)

scores1, scores2

({'rouge1': Score(precision=0.75, recall=0.75, fmeasure=0.75),
  'rougeL': Score(precision=0.75, recall=0.75, fmeasure=0.75)},
 {'rouge1': Score(precision=0.75, recall=0.75, fmeasure=0.75),
  'rougeL': Score(precision=0.5, recall=0.5, fmeasure=0.5)})

In [32]:
reference = ["police killed the gunman", "police kill the gunman"]
candidate1 = "police kill the gunman"
candidate2 = "the gunman kill police"

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=False)
scores1 = []
scores2 = []
for r in reference:
    scores1.append(scorer.score(r, candidate1)["rougeL"][0])

for r in reference:
    scores2.append(scorer.score(r, candidate2)["rougeL"][0])
scores1, scores2, max(scores1), max(scores2)

([0.75, 1.0], [0.5, 0.5], 1.0, 0.5)

In [28]:
scores1[0]["rougeL"][0]

0.75